# Flight Assignment

A small airlane company produces a weekly plan for the assignment of pilots and co-pilots to flights. In a pre-processing phase, the list of flights has been analysed to identify incompatible flights (those that cannot have the same crew members, like flights departing at the same hour).

Our company follows these rules:

- each flight needs a pilot and a co-pilot to depart.
- both pilots and co-pilots cannot exceed a weekly amount of flight hours.
- each pilot (resp. co-pilot) is paid based on several factors (e.g., the experience, the rank, flight hours)

The airplane company want to produce a weekly plan that minimizes the costs.

## Data

*P*: Set of pilots

*C*: Set of co-pliots

*V*: Set of flights

*I_v ⊆ V*: flights incompatibilities of flight *v ∈ V*

*f_(pv)*: Cost per hour of pilot *p ∈ P* if assigned to *v ∈ V*

*g_(cv)*: Cost per hour of pilot *c ∈ C* if assigned to *v ∈ V*

*q*: Weekly amount of flight hours for each pilot and co-pilot 

*t_v*: Flight hours of fliht *v ∈ V*

## Variables

*x_(pv)*: 1 if pilot *p* is assigned to flight *v*, and 0 otherwise

*y_(cv)*: 1 if co-pilot *c* is assigned to flight *v*, and 0 otherwise

## Objective Function

Minimize flight costs z^* = min ∑ _ (v ∈ V) t_v . ( ∑ _ (p ∈ P) f_(pv) + ∑ _ (c ∈ C) g_(cv) )

## Constraints

- One pilot and one co-pilot for each pilot

    ∑_(p ∈ P) x_(pv) = 1, ∀ *v ∈ V*
    
    ∑_(c ∈ C) y_(cv) = 1, ∀ *v ∈ V*
    

- Do not exceed flight hours for each pilot and co-pilot

    ∑ _ (v ∈ V) t_v * x_(pv) <= q, ∀ *p ∈ P*
    
    ∑ _ (v ∈ V) t_v * y_(cv) <= q, ∀ *c ∈ C*
    
    
- Avoid to assign the same crew to two incompatible flights

    x_(pv) + x_(pv') <= 1, ∀ *p ∈ P*, ∀ *v ∈ V*, ∀ *v' ∈ I_v*
    
    x_(cv) + x_(cv') <= 1, ∀ *c ∈ C*, ∀ *v ∈ V*, ∀ *v' ∈ I_v*

## Model Implementation

In [1]:
import pandas as pd

crew = pd.read_csv('crew.csv')
flights = pd.read_csv('flights.csv')
costs = pd.read_csv('costs.csv', index_col=0)

In [2]:
crew.head()

,Pilots,Co-pilots
0,Adriana C. Ocampo Uria,Cecilia Payne-Gaposchkin
1,Albert Einstein,Chien-Shiung Wu
2,Anna K. Behrensmeyer,Dorothy Hodgkin
3,Blaise Pascal,Edmond Halley
4,Caroline Herschel,Edwin Powell Hubble


In [3]:
flights.head()

,Flights,Time,ZT2898,ZT7947,ZT1277,ZT7456,ZT3554,ZT3234,ZT3821,ZT4311,ZT7579,ZT6742,ZT1557,ZT9968,ZT5269,ZT5845,ZT6743
0,ZT2898,3.0,ZT9968,ZT7456,ZT5845,ZT2898,ZT7947,ZT9968,ZT7579,ZT9968,ZT2898,ZT2898,ZT7947,ZT2898,ZT7947,ZT2898,ZT7947
1,ZT7947,1.0,ZT7947,ZT6743,ZT3821,ZT7579,ZT1277,ZT7579,ZT6742,ZT7947,ZT6742,ZT5845,ZT1277,ZT7947,ZT1277,ZT6743,ZT1277
2,ZT1277,1.5,ZT7456,ZT5269,ZT1557,ZT6743,ZT6743,ZT3821,ZT5269,ZT1277,ZT5845,ZT1277,ZT5269,ZT1277,ZT7456,ZT1277,ZT7456
3,ZT7456,4.0,ZT6742,ZT4311,NaN,ZT4311,ZT3234,ZT7456,ZT7456,ZT6743,ZT7456,ZT7456,ZT3554,NaN,NaN,ZT7456,ZT3554
4,ZT3554,2.0,ZT4311,ZT3554,NaN,ZT3554,NaN,ZT3554,ZT3554,ZT3554,ZT3554,NaN,NaN,NaN,NaN,NaN,ZT3234


In [4]:
costs.head()

,ZT2898,ZT7947,ZT1277,ZT7456,ZT3554,ZT3234,ZT3821,ZT4311,ZT7579,ZT6742,ZT1557,ZT9968,ZT5269,ZT5845,ZT6743
Pilots,,,,,,,,,,,,,,,
Adriana C. Ocampo Uria,42,49,46,42,49,34,32,41,45,44,43,42,49,37,47
Albert Einstein,38,48,48,46,35,44,47,43,31,42,40,47,47,33,49
Anna K. Behrensmeyer,34,43,42,36,30,30,39,42,44,48,48,36,41,42,39
Blaise Pascal,45,44,50,49,46,32,50,32,37,37,48,39,38,33,43
Caroline Herschel,49,50,50,30,42,32,30,49,47,44,30,46,31,44,48


## Data - Initializing Sets

In [5]:
import pyomo.environ as pyo

model = pyo.AbstractModel()

model.P = pyo.Set(initialize=crew['Pilots'].dropna().tolist())
model.C = pyo.Set(initialize=crew['Co-pilots'].dropna().tolist())
model.V = pyo.Set(initialize=flights['Flights'].tolist())

## Data - Flights Parameters

In [6]:
model.t = pyo.Param(model.V, initialize={row['Flights']: row['Time'] for i, row in flights.iterrows()})
model.I = pyo.Param(model.V, model.V, rule=lambda model, v1, v2: 1 if v1 in flights[v2].tolist() else 0)

## Data - Pilots Parameters

In [7]:
def read_costs(model, p, v):
    return costs.at[p, v]

model.f = pyo.Param(model.P, model.V, rule=read_costs)
model.g = pyo.Param(model.C, model.V, rule=read_costs)
model.q = pyo.Param(initialize=10)

## Variables

In [8]:
model.x = pyo.Var(model.P, model.V, within=pyo.Binary)
model.y = pyo.Var(model.C, model.V, within=pyo.Binary)

## Objective Function

In [9]:
def obj_functioin(model):
    return sum(model.t[v] * \
               (sum(model.f[p, v] * model.x[p, v] for p in  model.P) + \
               sum(model.g[c, v] * model.y[c, v] for c in model.C)) for v in model.V)

model.z = pyo.Objective(rule=obj_functioin, sense=pyo.minimize)

## Constraints - Assigments

In [10]:
def pilot_assign_cons(model, v):
    return sum(model.x[p, v] for p in model.P) == 1

def copilot_assign_cons(model, v):
    return sum(model.y[c, v] for c in model.C) == 1

model.pilot_assignment = pyo.Constraint(model.V, rule=pilot_assign_cons)
model.copilot_assignment = pyo.Constraint(model.V, rule=copilot_assign_cons)

## Constraints - Flights Incompatiilities

In [11]:
def pilot_incompatibility_cons(model, p, v1, v2):
    return (model.x[p, v1] + model.x[p, v2] <= 2 - model.I[v1, v2])

def copilot_incompatibility_cons(model, c, v1, v2):
    return (model.y[c, v1] + model.y[c, v2] <= 2 - model.I[v1, v2])

model.pilot_incompatibility = pyo.Constraint(model.P, model.V, model.V, rule=pilot_incompatibility_cons)
model.copilot_incompatibility = pyo.Constraint(model.C, model.V, model.V, rule=copilot_incompatibility_cons)

## Contraints - Flying Hours

In [12]:
def pilot_hours_cons(model, p):
    return sum(model.t[v] * model.x[p, v] for v in model.V) <= model.q

def copilot_hours_cons(model, c):
    return sum(model.t[v] * model.y[c, v] for v in model.V) <= model.q

model.pilot_hours = pyo.Constraint(model.P, rule=pilot_hours_cons)
model.copilot_hours = pyo.Constraint(model.C, rule=copilot_hours_cons)

## Solving the Model

In [13]:
instance = model.create_instance()
solver = pyo.SolverFactory('glpk')
results = solver.solve(instance)
print("Weekly plan cost {}".format(pyo.value(instance.z)))

Weekly plan cost 1630.5


In [14]:
print(results)


Problem: 
- Name: unknown
  Lower bound: 1630.5
  Upper bound: 1630.5
  Number of objectives: 1
  Number of constraints: 11331
  Number of variables: 751
  Number of nonzeros: 23251
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.05221891403198242
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



Print the crew people assigned to each flight

In [15]:
pd.DataFrame.from_records(
    [(v, p, c) for p in instance.P for c in instance.C for v in instance.V 
        if instance.x[p,v].value > 0 and instance.y[c,v].value > 0 ], 
        columns=['Flights', 'Pilot', 'Co-pilot'], index='Flights'
)

,Pilot,Co-pilot
Flights,,
ZT7456,Caroline Herschel,Stephen Hawking
ZT7947,Geraldine Seydoux,Niels Bohr
ZT1557,Jacqueline K. Barton,Rita Levi-Montalcini
ZT9968,Jocelyn Bell Burnell,Melissa Franklin
ZT7579,Jocelyn Bell Burnell,Wilhelm Conrad Roentgen
ZT5845,Johannes Kepler,Mildred S. Dresselhaus
ZT3554,Lene Vestergaard Hau,Rosalind Franklin
ZT2898,Lord Kelvin,Chien-Shiung Wu
ZT3821,Lord Kelvin,Chien-Shiung Wu


## Workers go on strike!

Labor unions declared a last minute strike, and some of the pilots and co-pilots of our company joined the strike.



In [16]:
strike = pd.read_csv("strike.csv")
strike

,Pilots
0,Frieda Robscheit-Robbins
1,Geraldine Seydoux
2,Gertrude B. Elion
3,Jacqueline K. Barton
4,Jocelyn Bell Burnell
5,Johannes Kepler
6,Lene Vestergaard Hau
7,Marie Curie
8,Max Born
9,Max Planck


We can use the same model and impose: 1) to avoid the assignment of flights to pilots on strike 2) to fix previously generated assignments in such a way that they are not changed

This can be done by fixing variables:

In [17]:
pilots_on_strike = strike['Pilots'].tolist()
for (p, v), var in instance.x.iteritems():
    if p in pilots_on_strike:
        var.value, var.fixed = 0, True
        
for (c, v), var in instance.y.iteritems():
    if c in pilots_on_strike:
        var.value, var.fixed = 0, True
        
for (p, v), var in instance.x.iteritems():
    if var.value > 0:
        var.fixed = True
        
for (c, v), var in instance.y.iteritems():
    if var.value > 0:
        var.fixed = True

In [18]:
results = solver.solve(instance)
print("Weekly plan cost {}".format(pyo.value(instance.z)))

Weekly plan cost 1664.5


In [19]:
pd.DataFrame.from_records(
    [(v, p, c) for p in instance.P for c in instance.C for v in instance.V 
        if instance.x[p,v].value > 0 and instance.y[c,v].value > 0 ], 
        columns=['Flights', 'Pilot', 'Co-pilot'], index='Flights'
)

,Pilot,Co-pilot
Flights,,
ZT7579,Albert Einstein,Wilhelm Conrad Roentgen
ZT3234,Anna K. Behrensmeyer,Patty Jo Watson
ZT4311,Blaise Pascal,Edmond Halley
ZT5845,Blaise Pascal,Mildred S. Dresselhaus
ZT7456,Caroline Herschel,Stephen Hawking
ZT7947,Ingrid Daubechies,Shannon W. Lucid
ZT1557,Jane Goodall,Edwin Powell Hubble
ZT3554,Lise Meitner,Ruzena Bajcsy
ZT9968,Lise Meitner,Wilhelm Conrad Roentgen


In [20]:
print(results)


Problem: 
- Name: unknown
  Lower bound: 1664.5
  Upper bound: 1664.5
  Number of objectives: 1
  Number of constraints: 11331
  Number of variables: 435
  Number of nonzeros: 13455
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.03251171112060547
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

